## Análise e Classificação de Reservas de Hotel  
Este notebook contém uma análise exploratória e um modelo de classificação aplicado a um conjunto de dados de reservas de hotel. O objetivo é entender os padrões presentes nas reservas e prever o status das reservas com base em diversas características.  
  
O notebook está dividido em várias seções, cada uma realizando uma etapa específica do processo de análise e modelagem:  
  
1) Preparação dos Dados: Carregamento dos dados, identificação de variáveis categóricas e pré-processamento.
2) Análise Exploratória: Exploração das características dos dados e visualização de padrões.
3) Engenharia de Recursos: Criação de variáveis dummy para características categóricas e codificação de rótulos.
4) Modelagem: Treinamento de um modelo de árvore de decisão para prever o status das reservas.
5) Avaliação do Modelo: Avaliação da precisão do modelo e análise da matriz de confusão.
  
Cada célula de código é acompanhada de comentários explicativos para facilitar a compreensão do processo realizado em cada etapa.  


* Importação das bibliotecas Pandas e NumPy.
* Leitura de um arquivo CSV chamado 'Hotel Reservations.csv' e armazenamento dos dados em um DataFrame chamado df.
* Exibição dos cinco primeiros registros do DataFrame.

In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('../Data/Hotel Reservations.csv', sep=',')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Hotel Reservations.csv'

* Iteração sobre as colunas do DataFrame.
* Para cada coluna, imprime o nome da coluna e o número de valores únicos presentes nela.

In [ ]:
for c in df.columns:
    print(f"{c:40s} - {df[c].nunique():>5d}")

* Remoção da coluna "Booking_ID" do DataFrame usando o método drop.
* Exibição do DataFrame após a remoção da coluna.

In [ ]:
df = df.drop("Booking_ID", axis = 1)
df

* Criação de variáveis dummy para as colunas categóricas do DataFrame usando a função `pd.get_dummies`.

In [ ]:
typeofmealplaDummies = pd.get_dummies(df["type_of_meal_plan"], prefix = "type_of_meal_plan")
roomtypereservedDummies = pd.get_dummies(df["room_type_reserved"], prefix = "room_type_reserved")
marketsegmenttypeDummies = pd.get_dummies(df["market_segment_type"], prefix = "market_segment_type")
requiredcarparkingspaceDummies = pd.get_dummies(df["required_car_parking_space"], prefix = "required_car_parking_space")

* Concatenação das variáveis dummy criadas anteriormente ao DataFrame original ao longo do eixo das colunas.

In [ ]:
df = pd.concat([df, typeofmealplaDummies], axis = 1)
df = pd.concat([df, roomtypereservedDummies], axis = 1)
df = pd.concat([df, marketsegmenttypeDummies], axis = 1)
df = pd.concat([df, requiredcarparkingspaceDummies], axis = 1)
df

* Codificação da coluna "_booking_status_" usando o LabelEncoder do `scikit-learn` para converter os rótulos em números inteiros.
* Exibição do DataFrame após a codificação.

In [ ]:
from sklearn.preprocessing import LabelEncoder
df["booking_status"] = LabelEncoder().fit_transform(df["booking_status"])
df

* Remoção das colunas "type_of_meal_plan", "room_type_reserved" e "market_segment_type" do DataFrame.

In [ ]:
df = df.drop("type_of_meal_plan", axis = 1)
df = df.drop("room_type_reserved", axis = 1)
df = df.drop("market_segment_type", axis = 1)

* Separação do DataFrame em conjuntos de características (X) e rótulos (y).
* Divisão dos conjuntos de treinamento e teste usando `train_test_split` do `scikit-learn`.

In [ ]:
X = df.drop("booking_status", axis = 1)
y = df["booking_status"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

* Cálculo da correlação entre cada característica e o rótulo ("booking_status").

In [ ]:
df.corr()['booking_status']

## Árvore de Decisão

Estamos utilizando o algoritmo Árvore de Decisão para realizar a classificação e avaliar o desempenho do modelo. A Árvore de Decisão é um algoritmo de aprendizado supervisionado que divide os dados em subconjuntos baseados em condições de atributos, criando uma estrutura em forma de árvore. Cada nó interno representa uma condição de atributo, cada ramo representa o resultado da condição, e cada folha representa uma classe ou valor final. Este método é intuitivo e fácil de interpretar, mas pode ser propenso a overfitting se não for devidamente podado.

* Inicialização e ajuste de um classificador de árvore de decisão aos dados de treinamento.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clfDT = DecisionTreeClassifier(criterion='gini',
                             splitter='best',
                             max_depth=None,
                             min_samples_split=2,
                             min_samples_leaf=1,
                             min_weight_fraction_leaf=0.0,
                             max_features=None,
                             random_state=42,
                             max_leaf_nodes=None,
                             min_impurity_decrease=0.0,
                             class_weight=None,
                             ccp_alpha=0.0,
                             monotonic_cst=None)
clfDT.fit(X_train, y_train)

* Cálculo dos indicadores do classificador usando os dados de teste.

In [ ]:
print(classification_report(y_test, clfDT.predict(X_test)))

* Cálculo e exibição da matriz de confusão usando os rótulos verdadeiros e preditos.

In [ ]:
from sklearn.metrics import confusion_matrix

cmDT = confusion_matrix(y_test, clfDT.predict(X_test))
print('Matriz de Confusão:')
print(cmDT)

## KNN

##### Nas células a seguir, estamos realizando classificação utilizando o algoritmo K-Vizinhos Mais Próximos (KNN) juntamente com a técnica de validação cruzada para avaliar o desempenho do modelo. O KNN é um algoritmo de aprendizado supervisionado que classifica os pontos de dados com base na classe majoritária dos seus vizinhos mais próximos.  

1) Importação de Bibliotecas: Primeiramente, importamos a classe `KNeighborsClassifier` do módulo `neighbors` do `scikit-learn`, além do `cross_val_score` para realizar validação cruzada e `matplotlib.pyplot` para visualização.
2) Inicialização do Classificador: Instanciamos o classificador KNN com os parâmetros padrão.
3) Validação Cruzada: Utilizamos a função `cross_val_score` para avaliar o desempenho do modelo através da validação cruzada. Essa função divide os dados em _k_ partes iguais e, para cada iteração, uma parte é usada como conjunto de teste enquanto as outras partes são usadas como conjunto de treinamento.
4) Avaliação do Desempenho: Calculamos a média dos escores de validação cruzada para avaliar o desempenho do modelo.
5) Visualização dos Resultados: Utilizamos `matplotlib.pyplot` para plotar um gráfico que mostra como o desempenho do modelo varia com o número de vizinhos.
Essas etapas nos ajudarão a entender como o desempenho do modelo KNN é afetado pelo número de vizinhos e a escolher o melhor valor para esse parâmetro.  
É recomendável que o valor de K não seja maior do que o numero de features usadas, portanto, vamos limitar a iteração ao número de colunas em _*X*_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
score_rate = []

for i in tqdm(range(1,X_train.shape[1], 2)):

    knn = KNeighborsClassifier(n_neighbors=i)
    accuracy = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy').mean()
    score_rate.append(accuracy)

plt.figure(figsize=(10,6))
plt.plot(range(1,X_train.shape[1], 2), score_rate, color='blue', linestyle='dashed', marker='o', markerfacecolor='red', markersize=10)

plt.title('Accuracy VS K Value')
plt.xlabel('K')
plt.ylabel('Accuracy')

* Usando o método do cotovelo, podemos ver que o ponto ótimo para treinamento é com 5 vizinhos.
* O próximo passo é a inicialização e ajuste de um classificador de KNN aos dados de treinamento.

In [ ]:
clfKNN = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    algorithm='auto',
    leaf_size=30,
    p=2,
    metric='minkowski',
    metric_params=None,
    n_jobs=None)

clfKNN.fit(X_train, y_train)

* Cálculo dos indicadores do classificador usando os dados de teste.

In [ ]:
print(classification_report(y_test, clfKNN.predict(X_test)))

* Cálculo e exibição da matriz de confusão usando os rótulos verdadeiros e preditos.

In [ ]:
cmKNN = confusion_matrix(y_test, clfKNN.predict(X_test))
print('Matriz de Confusão:')
print(cmKNN)

# Random Forest

Estamos utilizando o algoritmo Random Forest para realizar a classificação e avaliar o desempenho do modelo. O Random Forest é um algoritmo de aprendizado supervisionado que utiliza um conjunto de árvores de decisão, onde cada árvore é construída a partir de um subconjunto aleatório de dados e características. A classificação final é determinada pela votação majoritária das classes preditas por todas as árvores. Esta abordagem tende a melhorar a precisão do modelo e reduzir o risco de overfitting.

* Importação do módulo

In [ ]:
from sklearn.ensemble import RandomForestClassifier

* O próximo passo é a inicialização e ajuste de um classificador de random forest aos dados de treinamento.

In [ ]:
clfRF = RandomForestClassifier(random_state=11)

clfRF.fit(X_train, y_train)

* Cálculo dos indicadores do classificador usando os dados de teste.

In [ ]:
print(classification_report(y_test, clfRF.predict(X_test)))

* Cálculo e exibição da matriz de confusão usando os rótulos verdadeiros e preditos.

In [ ]:
cmRF = confusion_matrix(y_test, clfRF.predict(X_test))
print('Matriz de Confusão:')
print(cmRF)

## Conclusão
  
Com base nos scores fornecidos, parece que o modelo Decision Tree apresentou um desempenho superior ao modelo KNN para o problema específico em questão.  

* O _F1-score_ do modelo Decision Tree é mais alto do que o do modelo KNN para ambas as classes. Isso sugere que o modelo de árvore de decisão tem um melhor equilíbrio entre precisão e recall.
* Tanto os socres de _Recall_ quanto o de _Precision_ do modelo Decision Tree são mais altos do que os do modelo KNN para ambas as classes, mas a diferença não é muito significativa - o modelo KNN apresenta uma precisão um pouco mais baixa para a classe 0, mas uma precisão comparável para a classe 1. No entanto, o recall para a classe 0 é significativamente mais baixo em comparação com o modelo de árvore de decisão, enquanto o recall para a classe 1 é um pouco mais alto.
* A acurácia geral do modelo KNN é de 0.81, o que é um pouco menor do que a do modelo Decision Tree (0.87). No entanto, as diferenças não são tão significativas.
  
Com base nessas observações, podemos concluir que, para o conjunto de dados e o problema específico em questão, o modelo Decision Tree parece ser uma escolha ligeiramente melhor em termos de equilíbrio entre precisão, recall e acurácia.   
Importante ressaltar que em nenhum dos modelos, a parametrização foi alvo de estudo até o momento.  Trabalhos futuros podem considerar uma otimização como alvo de estudo, além do balanceamento do conjunto de treino/teste, utilizando técnicas de undersampling ou oversamplig, data augmentation ou SMOTE.
  
Além disso, outras conclusões podem ser destacadas:
* **Interpretabilidade**: Se for uma preocupação importante e o conjunto de dados não for extremamente grande, o modelo Decision Tree terá uma vantagem significativa.
* **Tempo de Treinamento**: o modelo Decision Tree geralmente levará menos tempo para treinar do que o modelo KNN.
* **Escalabilidade**: o KNN pode ser preferível, especialmente em conjuntos de dados menores ou com dimensionalidade moderada.